In [1]:
import wandb
import json
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)))
from utils import _scale_params, load_config

In [2]:
api = wandb.Api()
api.entity = "teresapelinski"
# faab_autoencoder_transformer/frraprup -- transformer
# faab_autoencoder_transformer_time/ehdzqdop -- transformer time

sweep_ids = ["pdwxlf94"]
sweeps = [api.sweep(f'teresapelinski/faab_autoencoder_transformer_time_jan/{sweep}') for sweep in sweep_ids]

_path = "../../models/trained/transformer-autoencoder-timecomp-jan"


In [3]:
selected_runs = []
for run in sweeps[0].runs:
    loss = run.summary.get("train_loss")
    if loss < 0.014:
        selected_runs.append(run)

In [4]:
selected_runs

[<Run teresapelinski/faab_autoencoder_transformer_time_jan/0daq0g9h (running)>,
 <Run teresapelinski/faab_autoencoder_transformer_time_jan/sys6hs5g (finished)>,
 <Run teresapelinski/faab_autoencoder_transformer_time_jan/8j4cqfd8 (finished)>]

In [5]:
# finished_runs = []

# # Iterate over runs in the sweep
# for run in sweep.runs:
#     # Fetch the run to get detailed information, including its state
#     detailed_run = wandb.Api().run(f"{run.project}/{run.id}")
#     # Check if the run is finished
#     if detailed_run.state == "finished":
#         finished_runs.append(detailed_run)


In [6]:
# import os

# #load api key from .env
# with open("../.env") as f:
#     for line in f:
#         if line.strip() and not line.startswith('#'):
#             key, value = line.strip().split('=', 1)
#             os.environ[key] = value


In [7]:
# wandb.login(key=os.environ["WANDB_API_KEY"])

In [8]:
id_ep = {}
run_ids = []
for run in selected_runs:
    files = [file.name for file in run.files()]
    matching_files = [file for file in files if file.endswith(".model")]
    epochs = [int(file.split("_")[-1].split(".model")[0]) for file in matching_files]
    if epochs == []:
        continue
    highest_epoch = max(epochs)
    id_ep[run.id] = highest_epoch
    run_ids.append(run.id)
    print(run.id, highest_epoch)    
    root_file_name = f'transformer_run_{run.id}_{highest_epoch}'
    run.file(f'{root_file_name}.model').download(root=_path, exist_ok=True)
    json.dump(run.config, open(f'{_path}/{root_file_name}.json', "w"))
    json.dump({"train_loss": run.summary["train_loss"]}, open(f'{_path}/{root_file_name}_metrics.json', "w"))

json.dump(run_ids, open(f'{_path}/run_ids.json', "w"))

0daq0g9h 125
sys6hs5g 150
8j4cqfd8 150


In [9]:
json.dump(id_ep, open(f'{_path}/id_ep.json', "w"))

In [10]:
_train_losses = {}
for _id in run_ids:
    _train_losses[_id] = json.load(open(f'{_path}/transformer_run_{_id}_{id_ep[_id]}_metrics.json'))["train_loss"]

models_ordered_by_asc_loss = list(dict(sorted(_train_losses.items(), key=lambda item: item[1])).keys())
filename = _path + "/models_ordered_by_asc_loss.json"

with open(filename, 'w') as file:
    json.dump(models_ordered_by_asc_loss, file)

In [11]:
scaled_model_coordinates = _scale_params(epochs=id_ep, path = _path)

filename = _path + "/scaled_params.json"
with open (filename, 'w') as file:
    json.dump(scaled_model_coordinates, file)

In [12]:
all_configs = {}
for run in run_ids:
    config = load_config(run, epochs=id_ep[run],path=_path)
    all_configs[run] = config
    
    

In [13]:
all_configs

{'0daq0g9h': {'mask': False,
  'pred': False,
  'model': 'transformer',
  'device': 'cuda:0',
  'epochs': 150,
  'dropout': 0.25875870056013905,
  'project': 'faab_autoencoder_transformer',
  'seq_len': 1024,
  'feat_len': 8,
  'criterion': 'weighted_mse',
  'num_heads': 2,
  'optimizer': 'rmsprop',
  'batch_size': 64,
  'num_layers': 1,
  'pickle_path': 'src/dataset/data/0117/0117_processed_1024.pkl',
  'comp_seq_len': 128,
  'ff_size_time': 1024,
  'weights_path': 'src/dataset/data/0117/weights.pkl',
  'comp_feat_len': 4,
  'learning_rate': 0.000224782346487706,
  'max_grad_norm': 0,
  'pe_scale_factor': 1,
  'plotter_samples': 5,
  'scheduler_gamma': 0.1,
  'ff_size_features': 64,
  'num_encoder_layers': 7,
  'scheduler_step_size': 0},
 'sys6hs5g': {'mask': False,
  'pred': False,
  'model': 'transformer',
  'device': 'cuda:0',
  'epochs': 150,
  'dropout': 0.1862820635724552,
  'project': 'faab_autoencoder_transformer',
  'seq_len': 1024,
  'feat_len': 8,
  'criterion': 'weighted_m